# NB 9: SQLite

The language for managing relational databases is the Structured Query Language, or SQL ("sequel").

In this class we will use the simplest, called [sqlite3](https://www.sqlite.org/). It stores the database in a simple file and can be run in a "standalone" mode from the command-line. All of the basic techniques in python apply to any commercial SQL backend.

Steps:
1. Create a connection to a database `.connect()` ~ not asked to do in this class
2. Create a cursor object to exceute commands on db `.cusor()`
3. Write Query (Create table, insert data) and use cursor to exceute command
4. Return results (mostly in pandas df for more manipulation)
5. Save exceuting to db using `.comit()`

In [31]:
import sqlite3 as db

# Connect to a database (or create one if it doesn't exist)
conn = db.connect('example.db')

# Create a 'cursor' for executing commands
c = conn.cursor()

**Important Usage Note:** If the named file does not yet exist, this code creates it. However, if the database has been created before, this same code will open it. This fact can be important when you are debugging. For example, if your code depends on the database not existing initially, then you may need to remove the file first. You're likely to encounter this situation in this notebook.

### Tables and Basic Queries
The central object of a relational database is a table, identical to a "tibble".

You can create a table to hold your data by using the command, `CREATE TABLE`.

Note: If you try to create a table that already exists, it will fail. If you are trying to carry out these exercises from scratch, you may need to remove any existing example.db file or destroy any existing table; you can do the latter with the SQL command, `DROP TABLE IF EXISTS`.

In [34]:
# If this is not the first time you run this cell, 
# you need to delete the existed "Students" table first
c.execute("DROP TABLE IF EXISTS Students")

# create a table named "Students" with 2 columns: "gtid" and "name".
# the type for column "gtid" is integer and for "name" is text. 
c.execute("CREATE TABLE Students (gtid INTEGER, name TEXT)")

To populate the table with items, you can use the command, [`INSERT INTO`](https://www.sqlite.org/lang_insert.html).

In [36]:
c.execute("INSERT INTO Students VALUES (123, 'Vuduc')")
c.execute("INSERT INTO Students VALUES (456, 'Chau')")
c.execute("INSERT INTO Students VALUES (381, 'Bader')")
c.execute("INSERT INTO Students VALUES (991, 'Sokol')")

The commands above modify the database. However, these are temporary modifications and aren't actually saved to the databases until you say so. The way to do that is to issue a commit operation from the connection object. Remember to encode commits when you intend for them to take effect.

In [38]:
conn.commit()

Another common operation is to perform a bunch of insertions into a table from a list of tuples. In this case, you can use `executemany()`.

In [40]:
more_students = [(723, 'Rozga'),
                 (882, 'Zha'),
                 (401, 'Park'),
                 (377, 'Vetter'),
                 (904, 'Brown')]

# '?' question marks are placeholders for the two columns in Students table
c.executemany('INSERT INTO Students VALUES (?, ?)', more_students)
conn.commit()

In [41]:
#Selects all rows from the Students table
c.execute("SELECT * FROM Students")
results = c.fetchall() #returns tuples, each tuple is a row

print("Your results:", len(results), "\nThe entries of Students:\n", results)

Your results: 9 
The entries of Students:
 [(123, 'Vuduc'), (456, 'Chau'), (381, 'Bader'), (991, 'Sokol'), (723, 'Rozga'), (882, 'Zha'), (401, 'Park'), (377, 'Vetter'), (904, 'Brown')]


**Exercise 1** (2 points). Suppose we wish to maintain a second table, called `Takes`, which records classes that students have taken and the grades they earn.

In particular, each row of `Takes` stores a student by his/her GT ID, the course he/she took, and the grade he/she earned in terms of GPA (i.e. 4.0, 3.0, etc). More formally, suppose this table is defined as follows:

In [43]:
c.execute('DROP TABLE IF EXISTS Takes')
c.execute('CREATE TABLE Takes (gtid INTEGER, course TEXT, grade REAL)')

Write a command to insert the following records into the Takes table.
```
Vuduc: CSE 6040 - A (4.0), ISYE 6644 - B (3.0), MGMT 8803 - D (1.0)
Sokol: CSE 6040 - A (4.0), ISYE 6740 - A (4.0)
Chau: CSE 6040 - A (4.0), ISYE 6740 - C (2.0), MGMT 8803 - B (3.0)
```
(Note: See students table above to get the GT IDs for Vuduc, Sokol, and Chau. You don't have to write any code to retrieve their GT IDs. You can just type them in manually. However, it would be a good and extra practice for you if you can use some sql commands to retrieve their IDs.)

In [45]:
###
### YOUR CODE HERE
###
# Insert Data
c.execute('INSERT INTO Takes (gtid, course, grade) VALUES (?,?,?)', (123,'CSE 6040', 4.0 ))
c.execute('INSERT INTO Takes (gtid, course, grade) VALUES (?,?,?)', (123,'ISYE 6644', 3.0 ))
c.execute('INSERT INTO Takes (gtid, course, grade) VALUES (?,?,?)', (123,'MGMT 8803', 1.0 ))
c.execute('INSERT INTO Takes (gtid, course, grade) VALUES (?,?,?)', (991,'CSE 6040', 4.0 ))
c.execute('INSERT INTO Takes (gtid, course, grade) VALUES (?,?,?)', (991,'ISYE 6740', 4.0 ))
c.execute('INSERT INTO Takes (gtid, course, grade) VALUES (?, ?, ?)', (456, 'CSE 6040', 4.0))
c.execute('INSERT INTO Takes (gtid, course, grade) VALUES (?, ?, ?)', (456, 'ISYE 6740', 2.0))
c.execute('INSERT INTO Takes (gtid, course, grade) VALUES (?, ?, ?)', (456, 'MGMT 8803', 3.0))
conn.commit()
#conn.close()

# Displays the results of your code
c.execute('SELECT * FROM Takes')
results = c.fetchall()
print("Your results:", len(results), "\nThe entries of Takes:", results)

Your results: 8 
The entries of Takes: [(123, 'CSE 6040', 4.0), (123, 'ISYE 6644', 3.0), (123, 'MGMT 8803', 1.0), (991, 'CSE 6040', 4.0), (991, 'ISYE 6740', 4.0), (456, 'CSE 6040', 4.0), (456, 'ISYE 6740', 2.0), (456, 'MGMT 8803', 3.0)]


Using `executemany()` This method allows you to insert multiple rows in a single query. It takes two arguments: the SQL query and a list of tuples containing the values to be inserted.

In [47]:
#Alternative Way using 
c.execute('DROP TABLE IF EXISTS Takes')
c.execute('CREATE TABLE Takes (gtid INTEGER, course TEXT, grade REAL)')

data = [
    (123, 'CSE 6040', 4.0),
    (123, 'ISYE 6644', 3.0),
    (123, 'MGMT 8803', 1.0),
    (991, 'CSE 6040', 4.0),
    (991, 'ISYE 6740', 4.0),
    (456, 'CSE 6040', 4.0),
    (456, 'ISYE 6740', 2.0),
    (456, 'MGMT 8803', 3.0)
]

# Insert data using executemany
c.executemany('INSERT INTO Takes (gtid, course, grade) VALUES (?, ?, ?)', data)
c.execute('SELECT * FROM Takes')
rows = c.fetchall()   #alternative way to iterate through results
for row in rows:
    print(row)

#connection.commit()
#connection.close()

(123, 'CSE 6040', 4.0)
(123, 'ISYE 6644', 3.0)
(123, 'MGMT 8803', 1.0)
(991, 'CSE 6040', 4.0)
(991, 'ISYE 6740', 4.0)
(456, 'CSE 6040', 4.0)
(456, 'ISYE 6740', 2.0)
(456, 'MGMT 8803', 3.0)


### Join Queries
The main type of query that combines information from multiple tables is the join query. Recall from our discussion of tibbles these four types:

```
INNER JOIN(A, B): Keep rows of A and B only where A and B match
OUTER JOIN(A, B): Keep all rows of A and B, but merge matching rows and fill in missing values with some default (NaN in Pandas, NULL in SQL)
LEFT JOIN(A, B): Keep all rows of A but only merge matches from B.
RIGHT JOIN(A, B): Keep all rows of B but only merge matches from A.
```

You can use the `WHERE` clause of a SELECT statement to specify how to match rows from the tables being joined. For example, recall that the Takes table stores classes taken by each student. However, these classes are recorded by a student's GT ID. Suppose we want a report where we want each student's name rather than his/her ID. We can get the matching name from the Students table. Here is a query to accomplish this matching:

When writing queries, we recomend you use the three single quote paradigm, so you don't have to escape characters. 

In [50]:
#view students table
c.execute('SELECT * FROM Students')
results = c.fetchall()
print(results,'\n')

#Join tables to see all tuples (name, course, grade)
query = ''' 
        SELECT Students.name, Takes.course, Takes.grade
        FROM Students, Takes
        WHERE Students.gtid = Takes.gtid
'''

for match in c.execute(query): #Alternative way to iterate through results
    print(match)

# c.execute(query)
# results = c.fetchall()
# display(results)

[(123, 'Vuduc'), (456, 'Chau'), (381, 'Bader'), (991, 'Sokol'), (723, 'Rozga'), (882, 'Zha'), (401, 'Park'), (377, 'Vetter'), (904, 'Brown')] 

('Vuduc', 'CSE 6040', 4.0)
('Vuduc', 'ISYE 6644', 3.0)
('Vuduc', 'MGMT 8803', 1.0)
('Chau', 'CSE 6040', 4.0)
('Chau', 'ISYE 6740', 2.0)
('Chau', 'MGMT 8803', 3.0)
('Sokol', 'CSE 6040', 4.0)
('Sokol', 'ISYE 6740', 4.0)


**Exercise 2** (2 points). Define a query to select the names and grades of students where the course is CSE 6040. The code below will execute your query and store the results in a list results1 of tuples, where each tuple is a (name, grade) pair; thus, you should structure your query to match this format.

In [52]:
# Define `query` with your query:
###
### YOUR CODE HERE
###
query = '''
        SELECT Students.name, Takes.grade
        FROM Students, Takes
        WHERE Students.gtid = Takes.gtid and Takes.course = 'CSE 6040'
'''

c.execute(query)
results1 = c.fetchall()
results1

[('Vuduc', 4.0), ('Sokol', 4.0), ('Chau', 4.0)]

In [53]:
#Alternative Way
query = '''
        SELECT Students.name, Takes.grade
        FROM Takes
        JOIN Students ON Takes.gtid = Students.gtid
        WHERE Takes.course = 'CSE 6040'
'''

c.execute(query)
results1 = c.fetchall()
results1

[('Vuduc', 4.0), ('Sokol', 4.0), ('Chau', 4.0)]

**Exercise 3** (2 points). Execute a LEFT JOIN that uses Students as the left table, Takes as the right table, and selects a student's name and course grade. Write your query as a string variable named query, which the subsequent code will execute.

In [55]:
# Define `query` string here:
###
### YOUR CODE HERE
###
query = '''
        SELECT Students.name, Takes.grade
        FROM Students 
        LEFT JOIN Takes ON Students.gtid = Takes.gtid
'''

# Executes your `query` string:
c.execute(query)
matches = c.fetchall()
for i, match in enumerate(matches):
    print(i, "->", match)

0 -> ('Vuduc', 1.0)
1 -> ('Vuduc', 3.0)
2 -> ('Vuduc', 4.0)
3 -> ('Chau', 2.0)
4 -> ('Chau', 3.0)
5 -> ('Chau', 4.0)
6 -> ('Bader', None)
7 -> ('Sokol', 4.0)
8 -> ('Sokol', 4.0)
9 -> ('Rozga', None)
10 -> ('Zha', None)
11 -> ('Park', None)
12 -> ('Vetter', None)
13 -> ('Brown', None)


### Aggreagation
Aggregation is a summary of information across multiple records. It typically ignores NULL values.

Useful SQL aggregators : `MIN`, `MAX`, `SUM`, `AVG`, and `COUNT`.

Because there are not a lot of aggregation functions in SQL, Pandas will be the preferred method to perform more advanced aggregations (like sd, median, etc) on the data.

In [76]:
#average GPA for each unique GT ID
query = '''
        SELECT gtid, AVG(grade)
        FROM Takes 
        GROUP BY gtid
'''

#Option 1: Result in Tuples
for match in c.execute(query):
    print(match)

#Option 2: Result in Pandas df
df = pd.read_sql_query(query, conn)
df

(123, 2.6666666666666665)
(456, 3.0)
(991, 4.0)


,gtid,AVG(grade)
0,123,2.666667
1,456,3.000000
2,991,4.000000


### Clean Up
It's good practice to shutdown the cursor and connection, the same way you close files.
```
c.close()
conn.close()
```

In [51]:
conn.close()

## Part 2: NYC 311 Calls

In [158]:
import pandas as pd
import sqlite3 as db

# Connect to a Database (or create one if it doesn't exist)
connection = db.connect('Pandas-sql.db')
c = connection.cursor()   # Create cursor for executing commands

# Create a Table named "NYC_Reports" with 3 columns.
c.execute("DROP TABLE IF EXISTS NYC_Reports") 

create_table_query = '''
    CREATE TABLE NYC_Reports (
        CreatedDate TEXT, 
        ComplaintType TEXT, 
        City TEXT)
'''
c.execute(create_table_query)
connection.commit()
print("Table created successfully!-------------------------------------------")


#Insert Data
insert_data_query = '''
    INSERT INTO NYC_Reports (CreatedDate, ComplaintType, City)
    VALUES (?, ?, ?)
'''

data = [
    ('2015-09-15 02:14:04.000000', 'Illegal Parking', 'NEW YORK'),
    ('2015-09-15 03:12:49.000000', 'Noise - Street/Sidewalk', 'NEW YORK'),
    ('2015-09-15 06:11:19.000000', 'Noise - Street/Sidewalk', 'NEW YORK'),
    ('2015-09-15 08:09:46.000000', 'Noise - Commercial', 'BRONX'),
    ('2015-09-15 12:09:18.000000', 'Homeless Person', 'NEW YORK'),
    ('2015-09-14 23:18:38.000000', 'HEAT/HOT WATER', 'brooklyn'),
    ('2015-09-15 18:14:04.000000', 'Illegal Parking', None), 
    ('2015-09-14 23:18:38.000000', 'HEAT/HOT WATER', 'BROOKLYN'),
    ('2015-09-15 01:37:35.000000', 'heat/hot water', 'STATEN ISLAND'),
    ('2015-09-15 15:19:21.000000', 'NOISE - Street/Sidewalk', 'Jamacia'),
    ('2015-09-14 10:40:35.000000', 'heat/hot water', 'staten island')
]

c.executemany(insert_data_query, data)
connection.commit()
print("Data inserted successfully!--------------------------------------------")

#You can get the reults of your query by executing and returning data in tuples OR a pandas df
# Option 1: Example query to verify table:
# c.execute('SELECT * FROM NYC_Reports LIMIT 5')  
# for row in c.fetchall():
#     print(row)

#Option 2: Create DF from the query and view the table
query = '''SELECT * FROM NYC_Reports'''
nyc_df = pd.read_sql_query (query, connection)
nyc_df.head()

Table created successfully!-------------------------------------------
Data inserted successfully!--------------------------------------------


,CreatedDate,ComplaintType,City
0,2015-09-15 02:14:04.000000,Illegal Parking,NEW YORK
1,2015-09-15 03:12:49.000000,Noise - Street/Sidewalk,NEW YORK
2,2015-09-15 06:11:19.000000,Noise - Street/Sidewalk,NEW YORK
3,2015-09-15 08:09:46.000000,Noise - Commercial,BRONX
4,2015-09-15 12:09:18.000000,Homeless Person,NEW YORK


In [146]:
#COUNT(value) vs COUNT(*)
count_all = pd.read_sql_query('''SELECT COUNT(*) FROM NYC_Reports''', connection)
print(count_all) #counts all rows

count = pd.read_sql_query('''SELECT COUNT(City) FROM NYC_Reports''', connection)
print(count) #ignores null rows

   COUNT(*)
0        11
   COUNT(City)
0           10


**Exercise 1** (2 points). Create a string, query, containing an SQL query that will return the number of `noise complaints` by type. The columns should be named type and freq, and the results should be sorted in descending order by freq. Also, since we know some complaints use an inconsistent case, for your function convert complaints to lowercase.

In [152]:
#Count how many times each Ttype of noise complaint occurs in desc order. 
#Look for case-insensitive substring matches.

del query #clear existing query
query = '''
    SELECT 
        LOWER(ComplaintType) AS type, 
        COUNT(*) AS freq
    FROM NYC_Reports
    WHERE ComplaintType LIKE '%noise%'
    GROUP BY type  
    ORDER BY freq DESC
'''
#must aaply lower() to group by clause also OR just use alias that has lower() already
df = pd.read_sql_query(query, connection)
df

,type,freq
0,noise - street/sidewalk,3
1,noise - commercial,1


**Exercise 2** (2 points). Create a string variable, query, that contains an SQL query that will return the top 3 cities with the largest number of complaints, in descending order. It should return a table with two columns, one named name holding the name of the city, and one named freq holding the number of complaints by that city.

Like complaint types, cities are not capitalized consistently. Therefore, standardize the city names by converting them to uppercase.e.

In [176]:
del query #clear existing query

query = '''
    SELECT UPPER(City) AS city, Count(*) AS freq
    FROM NYC_Reports
    GROUP BY UPPER(City)
    ORDER BY freq DESC
    LIMIT 3
'''

top_cities_df = pd.read_sql_query(query, connection)
top_cities_df

,city,freq
0,NEW YORK,4
1,STATEN ISLAND,2
2,BROOKLYN,2


Case-insensitive grouping: `COLLATE NOCASE`. Another way to carry out the preceding query in a case-insensitive way is to add a COLLATE NOCASE qualifier to the GROUP BY clause. Doesn't apply to all columns you need to specify COLLATE NOCASE after each column you want case-insensitive in group by clause. 

**Exercise 3** (1 point). Implement a function that takes a list of strings, str_list, and returns a single string consisting of each value, str_list[i], enclosed by double-quotes and separated by a comma-space delimiters. For example, if:
```
assert str_list == ['a', 'b', 'c', 'd']   assert strs_to_args(str_list) == '"a", "b", "c", "d```

Avoid manipulating the input str_list directly.




In [203]:
def strs_to_args(str_list):
    assert type (str_list) is list
    assert all ([type (s) is str for s in str_list])
    ###
    ### YOUR CODE HERE
    ###
    #goal = '"a", "b", "c", "d"'
    new_str = ''
    for i in str_list:
        new_str += (f'"{i}", ')
    
    return new_str[:-2]
    #return ",".join([f"'{i}'" for i in str_list])
    
strs_to_args(['a', 'b', 'c', 'd'])

'"a", "b", "c", "d"'

The point of the function above is to transform the list TOP_CITIES into a string that can be inserted into the SQL query. It creates a string like 'NEW YORK', 'LOS ANGELES', 'CHICAGO' that can be read by SQL unlike a list.

We want to this because in the next excercise, we will focus on filtering results based on a list of values (top cities). You can either create a function that joins the list into a comma seperated string or do this directly.

**Exercise 4** (3 points). Suppose we want to look at the number of complaints by type and by city for only the top cities, i.e., those in the list TOP_CITIES computed above. Execute an SQL query to produce a tibble named df_complaints_by_city with the variables {complaint_type, city_name, complaint_count}.

In your output DataFrame, convert all city names to uppercase and convert all complaint types to lowercase.

In [270]:
TOP_CITIES = list(top_cities_df['city']) #list of the top cities
###
### YOUR CODE HERE
###
del query
query = '''
    SELECT 
        LOWER(ComplaintType) AS complaint_type, 
        UPPER(City) AS city_name,
        COUNT(*) AS complaint_count
    FROM NYC_Reports
    WHERE city_name IN ({})
    GROUP BY complaint_type, city_name
'''.format(strs_to_args(TOP_CITIES))
#.format(','.join(f"'{city}'" for city in TOP_CITIES))
# IN ({}) is a placeholder for TOP_CITIES list; filtering

df_complaints_by_city = pd.read_sql_query(query, connection)
df_complaints_by_city

,complaint_type,city_name,complaint_count
0,heat/hot water,BROOKLYN,2
1,heat/hot water,STATEN ISLAND,2
2,homeless person,NEW YORK,1
3,illegal parking,NEW YORK,1
4,noise - street/sidewalk,NEW YORK,2


**Exercise 5** (2 points). Suppose we want to create a different stacked bar plot that shows, for each complaint type  t and city  c, the fraction of all complaints of type  t
(across all cities, not just the top ones) that occurred in city  c. Store your result in a dataframe named df_plot_fraction. It should have the same columns as df_plot, except that the complaint_count column should be replaced by one named complaint_frac, which holds the fractional values.

In [278]:
query = '''
    SELECT 
        LOWER(ComplaintType) AS complaint_type, 
        UPPER(City) AS city_name,
        COUNT(*) AS complaint_count
    FROM NYC_Reports
    GROUP BY complaint_type, city_name
'''

df_plot = pd.read_sql_query(query, connection)
df_plot

,complaint_type,city_name,complaint_count
0,heat/hot water,BROOKLYN,2
1,heat/hot water,STATEN ISLAND,2
2,homeless person,NEW YORK,1
3,illegal parking,None,1
4,illegal parking,NEW YORK,1
5,noise - commercial,BRONX,1
6,noise - street/sidewalk,JAMACIA,1
7,noise - street/sidewalk,NEW YORK,2


In [296]:
df_plot['complaint_frac'] = df_plot['complaint_count'] / df_plot['complaint_count']
df_plot_fraction = df_plot.drop('complaint_count', axis=1)
#del df_plot_fraction['complaint_count']

df_plot_fraction

,complaint_type,city_name,complaint_frac
0,heat/hot water,BROOKLYN,1.0
1,heat/hot water,STATEN ISLAND,1.0
2,homeless person,NEW YORK,1.0
3,illegal parking,None,1.0
4,illegal parking,NEW YORK,1.0
5,noise - commercial,BRONX,1.0
6,noise - street/sidewalk,JAMACIA,1.0
7,noise - street/sidewalk,NEW YORK,1.0


#### Dates and times in SQL
You can query against by comparing to strings of the form, YYYY-MM-DD hh:mm:ss.

Extract just the hour from the time stamp, using SQL's `strftime()`.

In [ ]:
#Extract hour
query = '''
  SELECT 
      CreatedDate, STRFTIME('%H', CreatedDate) AS Hour, 
      LOWER(ComplaintType)
    FROM NYC_Reports
    LIMIT 5
'''
df = pd.read_sql_query (query, connection)
df

**Exercise 6** (3 points). Construct a tibble called df_complaints_by_hour, which contains the total number of complaints during a given hour of the day. That is, the variables or column names should be {hour, count} where each observation is the total number of complaints (count) that occurred during a given hour.

Interpret hour as follows: when hour is 02, that corresponds to the open time interval [02:00:00, 03:00:00.0).

In [366]:
del query
query = '''
    SELECT 
        CreatedDate, 
        strftime('%H', CreatedDate) AS hour,
        COUNT(*) AS count
    FROM NYC_Reports
    GROUP BY hour
'''

df_complaints_by_hour = pd.read_sql_query(query, connection)
df_complaints_by_hour

,CreatedDate,hour,count
0,2015-09-15 01:37:35.000000,01,1
1,2015-09-15 02:14:04.000000,02,1
2,2015-09-15 03:12:49.000000,03,1
3,2015-09-15 06:11:19.000000,06,1
4,2015-09-15 08:09:46.000000,08,1
5,2015-09-14 10:40:35.000000,10,1
6,2015-09-15 12:09:18.000000,12,1
7,2015-09-15 15:19:21.000000,15,1
8,2015-09-15 18:14:04.000000,18,1
9,2015-09-14 23:18:38.000000,23,2


**Exercise 7** (2 points). What is the most common hour for noise complaints? Compute a tibble called df_noisy_by_hour whose variables are {hour, count} and whose observations are the number of noise complaints that occurred during a given hour. Consider a "noise complaint" to be any complaint string containing the word noise. Be sure to filter out any dates without an associated time, i.e., a timestamp of 00:00:00.000.

In [378]:
del query 
query = '''
    SELECT 
        strftime('%H', CreatedDate) AS hour,
        COUNT(*) as count
    FROM NYC_Reports
    WHERE 
        ComplaintType LIKE '%noise%' AND
        strftime('%H:%M:%f', CreatedDate) <> '00:00:00.000'
    GROUP BY hour
    ORDER BY count DESC
'''

df_noisy_by_hour = pd.read_sql_query(query, connection)
df_noisy_by_hour

,hour,count
0,15,1
1,08,1
2,06,1
3,03,1


## Part 2: Advandced SQL Topics

#### SQL String Manipulation
SQLite lacks Built-in support for regular expressions and Advanced string to datetime parsing functionailty. 

These limitations can force you to implement more creative solutions, often involving combinations of functions like TRIM, SUBSTR, INSTR, and REPLACE to achieve results that would be straightforward in other environments.

**Exercise 1**: (0 points)
parse_classification

Your task: define parse_classification as follows:

Activity: Extract and Separate Title Classification Code and Description

Return: query: a Python string containing a SQLite query. It should query the database to create a new DataFrame from the job_title table with the following columns:

```
title_classification - original title_classification column from the job_title table.
title_classification_desc - text portion of the title_classification 
title_classification_code - integer portion of the title_classification 
```

Requirements/steps:
- The database table you will need is named job_title.
- The column you will work with is title_classification.

*Hint: There are multiple ways to solve this exercise. The following SQLite functions may help, but not all are strictly necessary, SUBSTR, INSTR, and REPLACE.*

In [263]:
# Define demo
import sqlite3 as db
import pandas as pd

connection = db.connect('jobs_db')
cursor = connection.cursor()

cursor.execute("DROP TABLE IF EXISTS job_title")
cursor.execute('''CREATE TABLE job_title (title_classification TEXT)''')

#cursor.execute('''INSERT INTO job_title VALUES('Competitive-1')''')
data = [('Competitive-1',), ('Competitive-5',), ('Non-Competitive-5',), ('Non-Competitive-1',), ('Competitive-1',)]
cursor.executemany('''INSERT INTO job_title (title_classification) VALUES(?)''', data)
connection.commit()

query = '''SELECT * FROM job_title'''
job_title_df = pd.read_sql_query(query, connection)
job_title_df

,title_classification
0,Competitive-1
1,Competitive-5
2,Non-Competitive-5
3,Non-Competitive-1
4,Competitive-1


In [265]:
### Solution - Exercise 0  
def parse_classification():
    ###
    ### YOUR CODE HERE
    ###
    query = '''
        SELECT 
            title_classification,
            SUBTSR()
            SUBSTR()
        FROM job_title
    '''
    return query

### Demo function call
parse_classification_query = parse_classification()
parse_classification_df = pd.read_sql_query(parse_classification_query, connection)
display(parse_classification_df.head())

,title_classification
0,Competitive-1
1,Competitive-5
2,Non-Competitive-5
3,Non-Competitive-1
4,Competitive-1


### CASE Statements

`CASE` statement allows you to recode variables based on specific conditions. It works similarly to an **IF-THEN-ELSE** structure in programming languages. By using CASE, you can create new columns, transform existing values, or recategorize data based on **conditional logic** directly within your SQL queries.

**Example**: 

You want to create a new column called salary_band that assigns a salary band based on the experience_level. Here’s a simple example:

```
SELECT
    employee_id,
    experience_level,
    CASE
        WHEN experience_level = 'Junior' THEN 'Low'
        WHEN experience_level = 'Mid' THEN 'Medium'
        WHEN experience_level = 'Senior' THEN 'High'
        ELSE 'Unknown'
    END AS salary_band
FROM
    employees;
```

### Inline Subqueries and Common Table Expressions (CTEs)

When working with SQL, it's common to encounter situations where you need to perform calculations or data transformations based on grouped or filtered data. Two powerful tools for handling such scenarios are inline subqueries and Common Table Expressions (CTEs). Both approaches allow you to structure your queries in a way that breaks down complex operations into manageable steps, in slighly different ways.

An **Inline Subquery** is a query nested within another SQL query, typically within the SELECT, WHERE, or FROM clause. Inline subqueries allow you to perform calculations or filtering within the context of the outer query. They are particularly useful when you need to filter results based on aggregated values like maximum, minimum, or average. Inline subqueries are powerful, but they can become less efficient when the same calculation needs to be repeated multiple times, as each repetition can slow down the query execution.

A **Common Table Expression (CTE)**, introduced by the WITH clause, is a named temporary result set that you can reference within your main query. CTEs improve the readability and maintainability of SQL code, especially for complex queries, by allowing you to break down operations into logical steps CTEs are often preferred for their clarity and efficiency, particularly when the same calculation or transformation needs to be referenced multiple times within a query.mp